In [ ]:
# import required packages
from alpha_vantage.timeseries import TimeSeries
import json
from kafka import KafkaConsumer
from kafka import KafkaProducer
import json
from datetime import datetime
import logging
from kafka.structs import TopicPartition
import os.path

In [ ]:
# messages to Kafka Server
topic = 'Enter Topic Name'
bootstrap_servers = 'localhost:9092'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer=lambda x:json.dumps(x).encode('utf-8'))

In [ ]:
"""
    Find last TimeStamp for which data is already uploaded to Kfaka server
"""

# initialize last timestamp with a required starting value
last_timestamp=datetime.strptime('2021-01-01 00:00:00', "%Y-%m-%d %H:%M:%S")

# retrieve last time stamp from file if exists for given topic
if os.path.isfile(topic+'_timestamp.txt'):
    file = open(topic+"_timestamp.txt", "r") 
    last_timestamp=datetime.strptime(file.read(), "%Y-%m-%d %H:%M:%S")
else:
    # if file does not exists then find it from Kafka server data already uploaded using Kafka consumer for all partitions
    # maintain offset for all partitions so that every time we don't have to start from first position
    consumer = KafkaConsumer(bootstrap_servers=bootstrap_servers, group_id='test',
        consumer_timeout_ms=2000,
        value_deserializer=lambda x: json.loads(x.decode('utf-8')))
    partitions=consumer.partitions_for_topic(topic=topic)
    # iterate through all partitions of the topic to get last time stamp
    for p in partitions:
        print(topic + ' -> ' + str(p))
        offset=0
        if os.path.isfile(topic+'_offset_'+str(p)+'.txt'):
            # retrieve offset stored in file if exists set offset for the topic & partition
            file = open(topic+'_offset_'+str(p)+'.txt', "r") 
            offset=int(file.read())
        consumer = KafkaConsumer(bootstrap_servers=bootstrap_servers, group_id='test',
            consumer_timeout_ms=2000,
            value_deserializer=lambda x: json.loads(x.decode('utf-8')))
        tp = TopicPartition(topic=topic, partition=p)
        consumer.assign([tp])
        consumer.seek(tp, offset)   # you can set the offset you want to resume from.
        last_dt={}
        new_offset=0
        for msg in consumer:
            # the msg begins with the offset you set
            last_dt=msg.value
            rec_date=datetime.strptime(list(last_dt.keys())[0], "%Y-%m-%d %H:%M:%S")
            if  rec_date > last_timestamp:
                    last_timestamp=rec_date
            new_offset=new_offset+1
        # store new offset in file so that only few records are required to process to get last TimeStamp
        file=open(topic+'_offset_'+str(p)+'.txt','w')
        if new_offset-5 >= 0:
            new_offset=new_offset-5
        else:
            new_offset=0
        file.write(str(new_offset))
        file.close()

file=open(topic+'_timestamp.txt','w')
file.write(str(last_timestamp))
file.close()
print(last_timestamp)

In [ ]:
"""
Recursively goes through the Alpha Vantage API stock data after waiting for 5 minutes
Upload new data arrived using AV API & updating recording TimeStamp
"""

import time
# Define key & TimeSeries object to pull stock data using Alpha Vantage API
key = 'Enter API Key'
ts = TimeSeries(key, output_format='json')
while True:
    try:
        # get data using get_intraday function of AV API
        data, meta = ts.get_intraday('MSFT', interval='1min', outputsize='full')

        # set stock data in chronological order
        lst=sorted(data)
        data2={}
        for l in lst:
            data2[l]=data[l]
        # send data to Kafka server for which data point data is not sent
        for key in data2:
            rec_date=datetime.strptime(key, "%Y-%m-%d %H:%M:%S")
            print(rec_date)
            if rec_date > last_timestamp:
                dic={}
                dic[key]=data[key]
                producer.send(topic=topic, value=dic)
                last_timestamp=rec_date
                print(rec_date)
        # updating last time stamp in related topic time stamp file
        file=open(topic+'_timestamp.txt','w')
        file.write(str(last_timestamp))
        file.close()
        time.sleep(300)
    except Exception as e:
        logging.warning('exception while pulling datapoints from API / sending data to Kafka')

In [ ]:
# close Kafka producer & consumer objects used
producer.close()
consumer.close()